### 전체 지하철역 데이터 전처리(9호선은 없는듯 함)

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import scipy as sp
import scipy.stats
import statsmodels.api as sm
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import glob
import pandas as pd
set(sorted([f.name for f in mpl.font_manager.fontManager.ttflist]))
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

$SO_2$

In [2]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6850-4FA4

 C:\dss13\회귀분석 프로젝트 디렉터리

2020-06-16  오후 01:47    <DIR>          .
2020-06-16  오후 01:47    <DIR>          ..
2020-06-16  오전 11:57    <DIR>          .ipynb_checkpoints
2016-07-19  오후 02:05            86,180 2015day_1.csv
2020-06-11  오후 08:13        13,017,253 2015sub.csv
2017-03-22  오후 09:48            87,036 2016day_1.csv
2020-06-11  오후 08:13        13,055,897 2016sub.csv
2018-02-02  오후 12:38            87,067 2017day_1.csv
2020-06-11  오후 08:13        12,684,562 2017sub.csv
2019-02-03  오후 07:06            87,083 2018day_1.csv
2020-06-11  오후 08:13        11,462,196 2018sub.csv
2020-06-15  오후 05:20           636,859 20190710기준_서울시정류장정보(수정) (1).xlsx
2020-05-22  오전 10:15            87,003 2019day_1.csv
2020-06-11  오후 08:13        11,470,440 2019sub.csv
2020-06-15  오후 02:44           105,983 4647,4661번 서울교통공사 역별 주소 현황 및 전화번호.xlsx
2020-06-15  오후 02:11         2,999,104 air.csv
2020-06-15  오후 11:23         3,464,396 air_all.csv
2020-06-15  오후 02:1

In [3]:
path ='C:/dss13/회귀분석 프로젝트'
filenames = glob.glob(path + "/*sub.csv")

In [4]:
filenames

['C:/dss13/회귀분석 프로젝트\\2015sub.csv',
 'C:/dss13/회귀분석 프로젝트\\2016sub.csv',
 'C:/dss13/회귀분석 프로젝트\\2017sub.csv',
 'C:/dss13/회귀분석 프로젝트\\2018sub.csv',
 'C:/dss13/회귀분석 프로젝트\\2019sub.csv']

In [5]:
sub_2016=pd.DataFrame(pd.read_csv(filenames[1],encoding='euc-kr',thousands = ','))
sub_2017=pd.DataFrame(pd.read_csv(filenames[2],encoding='euc-kr',thousands = ','))
sub_2018=pd.DataFrame(pd.read_csv(filenames[3],encoding='euc-kr',thousands = ','))
sub_2019=pd.DataFrame(pd.read_csv(filenames[4],encoding='euc-kr',thousands = ','))

In [6]:
sub_2016.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2017.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2018.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2019.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']

In [7]:
sub=pd.concat([sub_2016,sub_2017,sub_2018,sub_2019])

In [8]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20
0,2016-01-01,150,서울역(150),하차,1753,1856,2438,2656,2134,1859
1,2016-01-01,150,서울역(150),승차,584,1144,1723,3200,3076,2519
2,2016-01-01,151,시청(151),하차,554,939,689,951,792,618
3,2016-01-01,151,시청(151),승차,110,191,298,1458,1499,1383
4,2016-01-01,152,종각(152),승차,234,296,335,2580,2627,2302
...,...,...,...,...,...,...,...,...,...,...
200799,2019-12-31,2825,신흥,하차,86,243,208,510,562,493
200800,2019-12-31,2826,수진,승차,542,813,436,436,395,245
200801,2019-12-31,2826,수진,하차,115,433,259,463,501,452
200802,2019-12-31,2827,모란,승차,342,482,253,375,418,242


In [9]:
# 행 개수 확인
len(sub_2016)+len(sub_2017)+len(sub_2018)+len(sub_2019)

803604

In [10]:
# index 재할당
sub.reset_index(inplace=True,drop=True)

#### 이미 int타입이므로 필요 없었음,,
sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']]=sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']].apply(pd.to_numeric)

In [11]:
sub.날짜=pd.to_datetime(sub['날짜'])

In [12]:
sub['year']=sub.날짜.apply(lambda x: x.year)
sub['month']=sub.날짜.apply(lambda x: x.month)

In [13]:
# 요일 찾기
import datetime
from datetime import date

t=['월','화','수','목','금','토','일']
sub["요일"]=sub['날짜'].apply(lambda x: t[x.weekday()])

In [17]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20,year,month,요일
1,2016-01-01,150,서울역(150),승차,584,1144,1723,3200,3076,2519,2016,1,금
3,2016-01-01,151,시청(151),승차,110,191,298,1458,1499,1383,2016,1,금
4,2016-01-01,152,종각(152),승차,234,296,335,2580,2627,2302,2016,1,금
7,2016-01-01,153,종로3가(153),승차,184,189,327,2273,1812,1412,2016,1,금
8,2016-01-01,154,종로5가(154),승차,84,149,251,1332,1153,858,2016,1,금
...,...,...,...,...,...,...,...,...,...,...,...,...,...
803594,2019-12-31,2823,남한산성입구(성남법원.검찰청),승차,1848,2562,1244,850,760,439,2019,12,화
803596,2019-12-31,2824,단대오거리,승차,1338,1724,863,683,767,473,2019,12,화
803598,2019-12-31,2825,신흥,승차,468,614,365,359,375,269,2019,12,화
803600,2019-12-31,2826,수진,승차,542,813,436,436,395,245,2019,12,화


In [18]:
sub.dtypes

날짜        datetime64[ns]
역번호                int64
역명                object
구분                object
7to8               int64
8to9               int64
9to10              int64
17to18             int64
18to19             int64
19to20             int64
year               int64
month              int64
요일                object
dtype: object

In [19]:
sub=sub[sub["구분"]=='승차']

```




```

### 2호선 퇴근시간만

In [20]:
# 2호선 퇴근시간만
sub2=sub[(200<sub["역번호"]) & (sub["역번호"]<300)].reset_index(drop=True)
sub2.drop(['7to8', '8to9', '9to10'],1,inplace=True)

In [21]:
sub2

,날짜,역번호,역명,구분,17to18,18to19,19to20,year,month,요일
0,2016-01-01,201,시청(201),승차,1007,949,796,2016,1,금
1,2016-01-01,202,을지로입구(202),승차,2409,2697,2260,2016,1,금
2,2016-01-01,203,을지로3가(203),승차,554,515,446,2016,1,금
3,2016-01-01,204,을지로4가(204),승차,397,306,232,2016,1,금
4,2016-01-01,205,동대문역사문화공원(205),승차,1124,1000,943,2016,1,금
...,...,...,...,...,...,...,...,...,...,...
73045,2019-12-31,246,신설동,승차,421,499,249,2019,12,화
73046,2019-12-31,247,도림천,승차,98,104,65,2019,12,화
73047,2019-12-31,248,양천구청,승차,367,411,194,2019,12,화
73048,2019-12-31,249,신정네거리,승차,501,544,343,2019,12,화


In [22]:
# 17~20시간대의 탑승객
sub2["17to20"]=sub2["17to18"]+sub2["18to19"]+sub2["19to20"]

In [23]:
# 역명에서 역번호 제외
sub2["역명"]=sub2.역명.str.split("(",expand=True)[0]

In [24]:
# # 왜 위에꺼는 [0]하면 나오는데 이건 안나옴?????????????????????
# # 둘다 똑같이 dataframe인데..
# # A.위는 0이 컬럼명이었음..
# sub2[0]

In [25]:
sub2.drop(["역번호","17to18","18to19","19to20"],1,inplace=True)

In [26]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2016-01-01,시청,승차,2016,1,금,2752
1,2016-01-01,을지로입구,승차,2016,1,금,7366
2,2016-01-01,을지로3가,승차,2016,1,금,1515
3,2016-01-01,을지로4가,승차,2016,1,금,935
4,2016-01-01,동대문역사문화공원,승차,2016,1,금,3067
...,...,...,...,...,...,...,...
73045,2019-12-31,신설동,승차,2019,12,화,1169
73046,2019-12-31,도림천,승차,2019,12,화,267
73047,2019-12-31,양천구청,승차,2019,12,화,972
73048,2019-12-31,신정네거리,승차,2019,12,화,1388


In [27]:
len(sub2.역명.unique()),sub2.역명.unique()

(51,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '신천', '종합운동장',
        '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대', '서울대입구',
        '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청', '당산',
        '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동', '도림천',
        '양천구청', '신정네거리', '용두', '잠실새내'], dtype=object))

In [28]:
# 신천이 잠실새내로 변경되어 두개의 이름 존재
sub2.loc[sub2['역명']=="신천",'역명']="잠실새내"

In [29]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2016-01-01,시청,승차,2016,1,금,2752
1,2016-01-01,을지로입구,승차,2016,1,금,7366
2,2016-01-01,을지로3가,승차,2016,1,금,1515
3,2016-01-01,을지로4가,승차,2016,1,금,935
4,2016-01-01,동대문역사문화공원,승차,2016,1,금,3067
...,...,...,...,...,...,...,...
73045,2019-12-31,신설동,승차,2019,12,화,1169
73046,2019-12-31,도림천,승차,2019,12,화,267
73047,2019-12-31,양천구청,승차,2019,12,화,972
73048,2019-12-31,신정네거리,승차,2019,12,화,1388


In [30]:
len(sub2.역명.unique()),sub2.역명.unique()

(50,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
        '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
        '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
        '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동',
        '도림천', '양천구청', '신정네거리', '용두'], dtype=object))

In [31]:
import numpy as np
np.sum(sub2.isnull())

날짜        0
역명        0
구분        0
year      0
month     0
요일        0
17to20    0
dtype: int64

#### 신설동라인같은 역 제거

In [32]:
out=["까치산","신정네거리","양천구청","도림천","용답","신답","용두","신설동"]

In [33]:
# 까치산만 없음
sub2[sub2.역명=="까치산"]

,날짜,역명,구분,year,month,요일,17to20


In [34]:
sub2=sub2[~sub2.역명.isin(out)]

In [35]:
sub2.reset_index(inplace=True,drop=True)

In [36]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2016-01-01,시청,승차,2016,1,금,2752
1,2016-01-01,을지로입구,승차,2016,1,금,7366
2,2016-01-01,을지로3가,승차,2016,1,금,1515
3,2016-01-01,을지로4가,승차,2016,1,금,935
4,2016-01-01,동대문역사문화공원,승차,2016,1,금,3067
...,...,...,...,...,...,...,...
62818,2019-12-31,홍대입구,승차,2019,12,화,21357
62819,2019-12-31,신촌,승차,2019,12,화,12963
62820,2019-12-31,이대,승차,2019,12,화,5107
62821,2019-12-31,아현,승차,2019,12,화,1998


In [37]:
len(sub2.역명.unique()),sub2.역명.unique()

(43,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
        '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
        '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
        '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로'], dtype=object))

### 2호선 데이터 공휴일 여부 표시

In [38]:
holi=pd.read_csv("holi.csv",encoding="euc-kr")

In [39]:
holi

,locdate,seq,isHoliday,dateName,year,week
0,20150101,1,Y,신정,2015,목
1,20150218,1,Y,설날,2015,수
2,20150219,1,Y,설날,2015,목
3,20150220,1,Y,설날,2015,금
4,20150301,1,Y,삼일절,2015,일
...,...,...,...,...,...,...
86,20190913,1,Y,추석,2019,금
87,20190914,1,Y,추석,2019,토
88,20191003,1,Y,개천절,2019,목
89,20191009,1,Y,한글날,2019,수


In [40]:
# seq 공휴일이 겹친경우 카운트 됨
holi[holi.seq==2].locdate

47    20171002
49    20171003
Name: locdate, dtype: int64

In [41]:
# 띠용 20171002는 seq 2 인데 한개 밖에 없눈데..
holi[(holi.locdate==20171002) | (holi.locdate==20171003)]

,locdate,seq,isHoliday,dateName,year,week
47,20171002,2,Y,임시공휴일,2017,월
48,20171003,1,Y,개천절,2017,화
49,20171003,2,Y,추석,2017,화


In [42]:
# isHoliday가 N인경우는 쉬는날이 아니었으므로 제거
holi=holi[holi.isHoliday!="N"]

In [43]:
holi.groupby("year").size()

year
2015    16
2016    17
2017    19
2018    18
2019    16
dtype: int64

In [44]:
holi.locdate=pd.to_datetime(holi.locdate,format="%Y%m%d")

In [45]:
# sub2.holi는 공휴일 -> 주말은 아님
sub2["holi"]=sub2["날짜"].isin(holi.locdate)

In [46]:
sub2[sub2["year"]==2017][sub2.holi==True]
# 1900개행이 아닌 1800개 --> 20171003이 개천절과 추석이 겹쳐있기 때문에 중복!

,날짜,역명,구분,year,month,요일,17to20,holi
15738,2017-01-01,시청,승차,2017,1,일,1901,True
15739,2017-01-01,을지로입구,승차,2017,1,일,7001,True
15740,2017-01-01,을지로3가,승차,2017,1,일,1772,True
15741,2017-01-01,을지로4가,승차,2017,1,일,709,True
15742,2017-01-01,동대문역사문화공원,승차,2017,1,일,3018,True
...,...,...,...,...,...,...,...,...
31170,2017-12-25,홍대입구,승차,2017,12,월,28046,True
31171,2017-12-25,신촌,승차,2017,12,월,14101,True
31172,2017-12-25,이대,승차,2017,12,월,3716,True
31173,2017-12-25,아현,승차,2017,12,월,1093,True


In [47]:
# 주말이거나 공휴일이면 holi가 True가 되도록 변경
sub2["holi"]=np.where((sub2["요일"].isin(["토","일"]))|(sub2["holi"]==True),True,False)

In [48]:
sub2[sub2.holi==True].head(60)

,날짜,역명,구분,year,month,요일,17to20,holi
0,2016-01-01,시청,승차,2016,1,금,2752,True
1,2016-01-01,을지로입구,승차,2016,1,금,7366,True
2,2016-01-01,을지로3가,승차,2016,1,금,1515,True
3,2016-01-01,을지로4가,승차,2016,1,금,935,True
4,2016-01-01,동대문역사문화공원,승차,2016,1,금,3067,True
5,2016-01-01,신당,승차,2016,1,금,1572,True
6,2016-01-01,상왕십리,승차,2016,1,금,779,True
7,2016-01-01,왕십리,승차,2016,1,금,2139,True
8,2016-01-01,한양대,승차,2016,1,금,578,True
9,2016-01-01,뚝섬,승차,2016,1,금,1008,True


#### 환승역 개수 매칭

In [228]:
tran=pd.read_csv("transfer.csv",encoding="euc-kr")

In [229]:
tran.tail()

,역,환승 역 개수
38,구의,0
39,역삼,0
40,구로디지털단지,0
41,잠실새내,0
42,성수,0


In [51]:
sub2=pd.merge(sub2, tran, how="left",left_on='역명 ', right_on="역")

In [52]:
del sub2["역"]

In [53]:
sub2.rename(columns={"환승 역 개수":"ctransfer"},inplace=True)

In [54]:
sub2.ctransfer=sub2.ctransfer.fillna(0)

In [55]:
sub2.ctransfer=sub2.ctransfer.astype('int')

In [56]:
sub2[sub2.역명=='당산']

,날짜,역명,구분,year,month,요일,17to20,holi,ctransfer
36,2016-01-01,당산,승차,2016,1,금,1464,True,1
79,2016-01-02,당산,승차,2016,1,토,3023,True,1
122,2016-01-03,당산,승차,2016,1,일,2378,True,1
165,2016-01-04,당산,승차,2016,1,월,6898,False,1
208,2016-01-05,당산,승차,2016,1,화,6314,False,1
...,...,...,...,...,...,...,...,...,...
62644,2019-12-27,당산,승차,2019,12,금,6820,False,1
62687,2019-12-28,당산,승차,2019,12,토,3467,True,1
62730,2019-12-29,당산,승차,2019,12,일,2258,True,1
62773,2019-12-30,당산,승차,2019,12,월,5541,False,1


sub2.to_csv("sub2.csv",index=False,encoding="euc-kr")

```





```

### 기상 데이터 전처리

In [57]:
tem_2019= pd.read_csv('2019day_1.csv',encoding='euc-kr')
tem_2018= pd.read_csv('2018day_1.csv',encoding='euc-kr')
tem_2017= pd.read_csv('2017day_1.csv',encoding='euc-kr')
tem_2016= pd.read_csv('2016day_1.csv',encoding='euc-kr')
tem_2015= pd.read_csv('2015day_1.csv',encoding='euc-kr')

In [58]:
tem_2019.columns[36]

'합계 일사량(MJ/m2)'

In [59]:
tem_2019.rename(columns={'합계 일사량(MJ/m2)':'합계 일사(MJ/m2)'},inplace=True)

In [60]:
tem_2019.columns[36]

'합계 일사(MJ/m2)'

In [61]:
# 2016년이 윤년이라 하루 더 있음
tem = pd.concat([tem_2019,tem_2018,tem_2017,tem_2016,tem_2015])

In [62]:
tem

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr),합계 일조 시간(hr)
0,108,2019-01-01,-5.0,-8.2,631,-0.6,1525.0,NaN,NaN,NaN,...,2.6,6.1,9.1,14.9,17.1,1.0,1.4,NaN,NaN,NaN
1,108,2019-01-02,-4.9,-8.8,802,0.2,1458.0,NaN,NaN,NaN,...,2.4,5.9,8.9,14.7,17.1,1.3,1.9,NaN,NaN,NaN
2,108,2019-01-03,-3.5,-8.4,726,3.2,1436.0,NaN,NaN,NaN,...,2.3,5.8,8.8,14.7,17.0,1.4,2.0,NaN,NaN,NaN
3,108,2019-01-04,-1.1,-6.2,757,4.1,1504.0,NaN,NaN,NaN,...,2.2,5.7,8.7,14.5,17.0,0.9,1.3,NaN,NaN,NaN
4,108,2019-01-05,-2.8,-5.5,2358,1.1,1521.0,NaN,NaN,NaN,...,2.2,5.5,8.6,14.5,16.9,1.4,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,108,2015-12-27,-5.7,-8.9,751,-1.0,1505.0,NaN,NaN,NaN,...,5.0,8.3,10.7,15.9,17.2,NaN,1.2,NaN,NaN,9.2
361,108,2015-12-28,-5.7,-9.5,631,-1.0,1558.0,NaN,NaN,NaN,...,4.4,8.1,10.6,15.8,17.2,NaN,1.6,NaN,NaN,9.2
362,108,2015-12-29,-3.2,-8.7,739,2.9,1549.0,NaN,NaN,NaN,...,4.0,7.9,10.5,15.8,17.2,NaN,1.3,NaN,NaN,7.6
363,108,2015-12-30,0.3,-3.5,608,4.3,1425.0,3.75,NaN,NaN,...,3.8,7.7,10.3,15.7,17.2,NaN,0.2,2.5,NaN,3.1


In [63]:
tem.reset_index(drop=True,inplace=True)

In [64]:
np.where(tem.columns.isin(['일시','평균기온(°C)','평균 상대습도(%)','일강수량(mm)','합계 일사(MJ/m2)']))

(array([ 1,  2, 12, 24, 36], dtype=int64),)

In [65]:
tem=tem.iloc[:,np.where(tem.columns.isin(['일시','평균기온(°C)','평균 상대습도(%)','일강수량(mm)','합계 일사(MJ/m2)']))[0]]

In [66]:
tem.head()

,일시,평균기온(°C),일강수량(mm),평균 상대습도(%),합계 일사(MJ/m2)
0,2019-01-01,-5.0,NaN,49.5,7.84
1,2019-01-02,-4.9,NaN,42.8,10.48
2,2019-01-03,-3.5,NaN,38.8,10.28
3,2019-01-04,-1.1,NaN,55.5,6.20
4,2019-01-05,-2.8,NaN,40.3,10.05


In [67]:
tem.columns=["date","avgtemp","avgrainfall","avghimidity","solar"]

In [68]:
tem[tem.solar.isnull()]

,date,avgtemp,avgrainfall,avghimidity,solar
239,2019-08-28,26.1,NaN,66.2,NaN
240,2019-08-29,23.4,36.9,77.1,NaN
382,2018-01-18,2.1,0.0,59.6,NaN
390,2018-01-26,-14.8,NaN,34.9,NaN
410,2018-02-15,-0.7,NaN,28.0,NaN
835,2017-04-16,18.7,NaN,52.9,NaN
844,2017-04-25,16.0,NaN,56.8,NaN
1038,2017-11-05,8.4,NaN,52.5,NaN
1053,2017-11-20,-0.4,0.0,55.3,NaN
1060,2017-11-27,2.5,NaN,53.0,NaN


In [69]:
tem.date=pd.to_datetime(tem['date'])
tem['year']=tem.date.apply(lambda x: x.year)
tem['month']=tem.date.apply(lambda x: x.month)

In [70]:
msolar=tem.groupby(['year','month']).mean().reset_index()

In [71]:
a=np.where(tem.solar.isnull())[0]

In [72]:
for i in a:
    tem.solar[i]=msolar.solar[(msolar.year==tem.year[i]) & (msolar.month==tem.month[i])]

In [73]:
tem.iloc[a,:]

,date,avgtemp,avgrainfall,avghimidity,solar,year,month
239,2019-08-28,26.1,NaN,66.2,16.766897,2019,8
240,2019-08-29,23.4,36.9,77.1,16.766897,2019,8
382,2018-01-18,2.1,0.0,59.6,5.482414,2018,1
390,2018-01-26,-14.8,NaN,34.9,5.482414,2018,1
410,2018-02-15,-0.7,NaN,28.0,11.454815,2018,2
835,2017-04-16,18.7,NaN,52.9,17.461071,2017,4
844,2017-04-25,16.0,NaN,56.8,17.461071,2017,4
1038,2017-11-05,8.4,NaN,52.5,5.590000,2017,11
1053,2017-11-20,-0.4,0.0,55.3,5.590000,2017,11
1060,2017-11-27,2.5,NaN,53.0,5.590000,2017,11


In [74]:
tem.avgrainfall=tem['avgrainfall'].fillna(0)

In [75]:
tem.tail(5)

,date,avgtemp,avgrainfall,avghimidity,solar,year,month
1821,2015-12-27,-5.7,0.0,39.0,9.75,2015,12
1822,2015-12-28,-5.7,0.0,38.4,9.78,2015,12
1823,2015-12-29,-3.2,0.0,47.6,7.16,2015,12
1824,2015-12-30,0.3,2.5,67.6,5.43,2015,12
1825,2015-12-31,0.7,0.0,81.3,3.98,2015,12


```





```

## 따릉이

### 파일 다운

In [76]:
# csv파일 다운

In [77]:
path ='C:/Code/regression/따릉이'
filenames = glob.glob(path + "/서울특별시*.csv")

In [78]:
len(filenames)

0

In [79]:
bike1=pd.DataFrame()
bike2=pd.DataFrame()
bike3=pd.DataFrame()
bike4=pd.DataFrame()

In [80]:
a1=0
for i in range(0,11):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='euc-kr',thousands = ','))
    a1+=len(bike_data)
    print(filenames[i],i,a1)
    print(bike_data.columns[1:3])
    bike_data=bike_data.iloc[:,[1,2]]
    bike_data.columns=['대여일시','대여소번호']
    bike1=pd.concat([bike1,bike_data])

IndexError: list index out of range

In [81]:
bike1

""


In [82]:
a2=0
for i in range(11,15):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='euc-kr',thousands = ','))
    a2+=len(bike_data)
    print(filenames[i],i,a2)
    print(bike_data.columns[1:3])
    bike_data=bike_data.iloc[:,[1,2]]
    bike_data.columns=['대여일시','대여소번호']
    bike2=pd.concat([bike2,bike_data])

IndexError: list index out of range

In [83]:
bike2

""


In [84]:
a3=0
for i in range(15,30):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='cp949',thousands = ','))
    a3+=len(bike_data)
    print(filenames[i],i,a3)
    print(bike_data.columns[1:3])
    bike_data=bike_data.iloc[:,[1,2]]
    bike_data.columns=['대여일시','대여소번호']
    bike3=pd.concat([bike3,bike_data])

IndexError: list index out of range

In [85]:
bike3

""


In [86]:
a4=0
for i in range(30,38):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='cp949',thousands = ','))
    a4+=len(bike_data)
    print(filenames[i],i,a4)
    print(bike_data.columns[1:3])
    bike_data=bike_data.iloc[:,[1,2]]
    bike_data.columns=['대여일시','대여소번호']
    bike4=pd.concat([bike4,bike_data])

IndexError: list index out of range

In [87]:
bike4

""


In [88]:
# xlsx파일 다운

In [89]:
path ='C:/Code/regression/따릉이'
filenames = glob.glob(path + "/서울특별시*.xlsx")

In [90]:
len(filenames)

0

In [91]:
filenames

[]

In [92]:
bike5=pd.DataFrame()

In [93]:
a5=0
for i in range(len(filenames)):
    bike_data=pd.DataFrame(pd.read_excel(filenames[0],sheet_name="Sheet1",encoding='euc-kr',thousands = ','))
    a5+=len(bike_data)
    print(filenames[i],i,a5)
    print(bike_data.columns[1:3])
    bike_data=bike_data.iloc[:,[1,2]]
    bike_data.columns=['대여일시','대여소번호']
    bike5=pd.concat([bike5,bike_data])

In [94]:
bike5

""


In [95]:
bike1[bike1["대여소번호"].isnull()]

KeyError: '대여소번호'

In [96]:
bike2[bike2["대여소번호"].isnull()]

KeyError: '대여소번호'

In [97]:
bike3[bike3["대여소번호"].isnull()]

KeyError: '대여소번호'

In [98]:
bike4[bike4["대여소번호"].isnull()]

KeyError: '대여소번호'

In [99]:
bike5[bike5["대여소번호"].isnull()]

KeyError: '대여소번호'

bike1.to_csv("bike1.csv",index=False,encoding="euc-kr")
bike2.to_csv("bike2.csv",index=False,encoding="euc-kr")
bike3.to_csv("bike3.csv",index=False,encoding="euc-kr")
bike4.to_csv("bike4.csv",index=False,encoding="euc-kr")
bike5.to_csv("bike5.csv",index=False,encoding="euc-kr")

### bike전처리

In [100]:
bike1=pd.read_csv("bike1.csv",encoding='euc-kr',thousands = ',')
bike2=pd.read_csv("bike2.csv",encoding='euc-kr',thousands = ',')
bike3=pd.read_csv("bike3.csv",encoding='euc-kr',thousands = ',')
bike4=pd.read_csv("bike4.csv",encoding='euc-kr',thousands = ',')
bike5=pd.read_csv("bike5.csv",encoding='euc-kr',thousands = ',')

FileNotFoundError: [Errno 2] No such file or directory: 'bike1.csv'

In [101]:
# 특수기호 제외

In [102]:
bike1["대여일시"]=bike1["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike2["대여일시"]=bike2["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike3["대여일시"]=bike3["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike4["대여일시"]=bike4["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
# bike5["대여일시"]=bike5["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
# bike5이미 타임 스탬프 형식

KeyError: '대여일시'

In [103]:
bike1["대여소번호"]=bike1["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
bike2["대여소번호"]=bike2["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
# bike3["대여소번호"]=bike3["대여소번호"].str.replace(pat=r'[^0-9]', repl= r'', regex=True)
# bike4["대여소번호"]=bike4["대여소번호"].str.replace(pat=r'[^0-9]', repl= r'', regex=True)
# bike5["대여소번호"]=bike5["대여소번호"].str.replace(pat=r'[^0-9]', repl= r'', regex=True)

KeyError: '대여소번호'

In [104]:
# 대여일시 문자형을 datetime형태로

In [105]:
bike1['대여일시']=pd.to_datetime(bike1['대여일시'])
bike2['대여일시']=pd.to_datetime(bike2['대여일시'])
bike3['대여일시']=pd.to_datetime(bike3['대여일시'])
bike4['대여일시']=pd.to_datetime(bike4['대여일시'])
bike5['대여일시']=pd.to_datetime(bike5['대여일시'])

KeyError: '대여일시'

In [106]:
bike1[bike1["대여소번호"]=='중랑센터']

KeyError: '대여소번호'

```









```

## 대기질

In [107]:
air_2019=pd.read_csv("air_2019.csv",encoding='euc-kr')
air_2016=pd.read_excel("air_2016.xlsx",encoding='euc-kr')
air_2017=pd.read_excel("air_2017.xlsx",encoding='euc-kr')
air_2018=pd.read_excel("air_2018.xlsx",encoding='euc-kr')

FileNotFoundError: [Errno 2] No such file or directory: 'air_2019.csv'

In [108]:
air_2016.columns=air_2019.columns

NameError: name 'air_2019' is not defined

In [109]:
air_2016

NameError: name 'air_2016' is not defined

In [110]:
from datetime import datetime
from datetime import timedelta

In [111]:
date=[]
start = datetime(2018, 1, 1)
for i in range(365):
    date.append(start.strftime("%Y%m%d"))
    start=start+timedelta(days=1)

In [112]:
air_date=pd.DataFrame(air_2018["msrdt_de"].unique()).astype('str')

NameError: name 'air_2018' is not defined

In [113]:
# 누락된 날짜
set(date)-set(air_date[0])

NameError: name 'air_date' is not defined

In [114]:
# 누락된 날짜
set(date_2019)-set(air_date[0])

NameError: name 'date_2019' is not defined

In [115]:
air=pd.concat([air_2016,air_2017,air_2018,air_2019])

NameError: name 'air_2016' is not defined

In [116]:
air.reset_index(inplace=True,drop=True)

NameError: name 'air' is not defined

In [117]:
air.to_csv('air.csv',index=False,encoding="euc-kr")

NameError: name 'air' is not defined

In [118]:
import pandas as pd

#### 버스정류장갯수

In [515]:
bus2 = pd.read_csv('버스정류장진짜.csv',encoding='euc-kr')

In [516]:
bus2.tail()

,정류소번호,정류소명,X좌표,Y좌표
11275,25995,우성아파트,127.139339,37.550386
11276,25996,우성아파트,127.140046,37.550643
11277,25997,조일약국,127.123596,37.533630
11278,25998,성내시장,127.125497,37.536155
11279,25999,천호우체국.로데오거리,127.127337,37.540343


In [517]:
from haversine import haversine

for i in range(len(bus)):
    
    lyon = (45.7597, 4.8422) # (lat, lon)
    paris = (48.8567, 2.3508)

haversine(lyon, paris)


392.2172595594006

In [518]:
regis = pd.read_csv('역별주소2.csv',encoding='euc-kr')
regis

,전철역코드,전철역명,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS)
0,2818,가락시장,8,817,2818,525992.0,1108579.0,37.492522,127.118234
1,340,가락시장,3,350,2818,525992.0,1108579.0,37.492522,127.118234
2,2535,종로3가,5,534,153,498060.0,1130332.0,37.571607,126.991806
3,319,종로3가,3,329,153,498060.0,1130332.0,37.571607,126.991806
4,153,종로3가,1,130,153,498060.0,1130332.0,37.571607,126.991806
...,...,...,...,...,...,...,...,...,...
924,4505,동백,E,Y114,4505,533700.0,1046500.0,37.269043,127.152716
925,4506,초당,E,Y115,4506,535500.0,1044100.0,37.260752,127.159443
926,4508,삼가,E,Y116,4508,537300.0,1039000.0,37.242115,127.168075
927,4509,시청?용인대,E,Y117,4509,539500.0,1038300.0,37.239151,127.178406


In [519]:
regis2=regis[regis['호선']=='2']

In [520]:
regis2.tail(5)

,전철역코드,전철역명,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS)
362,248,양천구청,2,234-2,248,470185.0,1113852.0,37.512398,126.865819
363,249,신정네거리,2,234-3,249,467370.0,1116015.0,37.520074,126.852912
364,250,용두,2,211-3,250,508200.0,1131077.0,37.574028,127.038091
388,223,교대,2,223,330,502900.0,1108655.0,37.493415,127.014080
677,260,까치산,2,234-4,2519,466017.0,1119387.0,37.531768,126.846683


In [521]:
regis2.rename(columns={"전철역명":"역명"},inplace=True)
regis2=regis2[~regis2.역명.isin(out)]

In [522]:
regis2['역명'].value_counts()

왕십리          4
홍대입구         3
동대문역사문화공원    3
을지로3가        2
사당           2
신당           2
을지로4가        2
시청           2
종합운동장        2
선릉           2
충정로          2
교대           2
대림           2
건대입구         2
영등포구청        2
합정           2
신도림          2
강남           2
당산           2
잠실           2
잠실나루         1
삼성           1
신림           1
방배           1
역삼           1
문래           1
뚝섬           1
성수           1
신대방          1
서울대입구        1
을지로입구        1
구의           1
구로디지털단지      1
이대           1
봉천           1
아현           1
신촌           1
잠실새내         1
강변           1
서초           1
상왕십리         1
한양대          1
낙성대          1
Name: 역명, dtype: int64

In [523]:
regis2=regis2.drop_duplicates()

In [524]:
regis2.count()

전철역코드       43
역명          43
호선          43
외부코드        43
사이버스테이션     43
X좌표         43
Y좌표         43
X좌표(WGS)    43
Y좌표(WGS)    43
dtype: int64

In [525]:
regis2['역명'].value_counts()

을지로3가        1
시청           1
신당           1
을지로입구        1
서울대입구        1
신대방          1
잠실나루         1
잠실           1
뚝섬           1
왕십리          1
역삼           1
을지로4가        1
종합운동장        1
방배           1
삼성           1
동대문역사문화공원    1
신림           1
홍대입구         1
신도림          1
문래           1
구의           1
성수           1
구로디지털단지      1
잠실새내         1
한양대          1
당산           1
강남           1
상왕십리         1
합정           1
영등포구청        1
서초           1
강변           1
신촌           1
이대           1
건대입구         1
대림           1
사당           1
교대           1
충정로          1
아현           1
선릉           1
봉천           1
낙성대          1
Name: 역명, dtype: int64

In [526]:
regis2.drop(['전철역코드','호선', '외부코드', '사이버스테이션','X좌표','Y좌표'],1,inplace=True)

In [527]:
regis2['busstation300m']=0
regis2['busstation500m']=0

regis2

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m
10,동대문역사문화공원,37.565138,127.007896,0,0
17,신도림,37.508725,126.891295,0,0
21,왕십리,37.561533,127.037732,0,0
25,시청,37.564718,126.977108,0,0
27,교대,37.493415,127.014080,0,0
32,강변,37.535095,127.094681,0,0
39,을지로3가,37.566295,126.991910,0,0
42,홍대입구,37.557192,126.925381,0,0
51,영등포구청,37.524970,126.895951,0,0
58,을지로4가,37.566941,126.998079,0,0


In [528]:
regis2.rename(columns={"전철역명":"역명"},inplace=True)
regis2=regis2[~regis2.역명.isin(out)]


In [529]:
regis2.rename(columns={"전철역명":"역명"},inplace=True)

In [530]:
set(sub2['역명'].unique())-set(regis2['역명'].unique())

set()

In [531]:
regis2.head(5)

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m
10,동대문역사문화공원,37.565138,127.007896,0,0
17,신도림,37.508725,126.891295,0,0
21,왕십리,37.561533,127.037732,0,0
25,시청,37.564718,126.977108,0,0
27,교대,37.493415,127.014080,0,0


In [532]:
bus3 = bus2[['X좌표','Y좌표']]

In [533]:
bus3.iloc[:,0]

0        126.987750
1        126.996566
2        126.998340
3        126.987613
4        127.001744
            ...    
11275    127.139339
11276    127.140046
11277    127.123596
11278    127.125497
11279    127.127337
Name: X좌표, Length: 11280, dtype: float64

In [534]:
bus3

,X좌표,Y좌표
0,126.987750,37.569765
1,126.996566,37.579183
2,126.998340,37.582671
3,126.987613,37.568579
4,127.001744,37.586243
...,...,...
11275,127.139339,37.550386
11276,127.140046,37.550643
11277,127.123596,37.533630
11278,127.125497,37.536155


In [535]:
bus3.dtypes

X좌표    float64
Y좌표    float64
dtype: object

In [536]:
regis2.iloc[0,1],regis2.iloc[0,2]

(37.565138, 127.007896)

In [537]:
regis2

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m
10,동대문역사문화공원,37.565138,127.007896,0,0
17,신도림,37.508725,126.891295,0,0
21,왕십리,37.561533,127.037732,0,0
25,시청,37.564718,126.977108,0,0
27,교대,37.493415,127.014080,0,0
32,강변,37.535095,127.094681,0,0
39,을지로3가,37.566295,126.991910,0,0
42,홍대입구,37.557192,126.925381,0,0
51,영등포구청,37.524970,126.895951,0,0
58,을지로4가,37.566941,126.998079,0,0


In [538]:
bus3=bus3[['Y좌표','X좌표']]

In [539]:
bus3.rename(columns={"Y좌표":"X좌표","X좌표":"Y좌표"},inplace=True)

In [540]:
bus3

,X좌표,Y좌표
0,37.569765,126.987750
1,37.579183,126.996566
2,37.582671,126.998340
3,37.568579,126.987613
4,37.586243,127.001744
...,...,...
11275,37.550386,127.139339
11276,37.550643,127.140046
11277,37.533630,127.123596
11278,37.536155,127.125497


In [541]:
import folium
import json
m = folium.Map(location=[37.5051,126.97], zoom_start=12) 

for i in regis2.index[:1000]:
    folium.Circle(
        location = regis2.loc[i, ['X좌표(WGS)', 'Y좌표(WGS)']],
        tooltip = regis2.loc[i, 'busstation300m'],
        radius = 300   ).add_to(m) 
m

In [542]:
from haversine import haversine
for i in range(0,len(regis2)):
    for j in range(0,len(bus3)):
        레지스=(regis2.iloc[i,1],regis2.iloc[i,2])
        버스=(bus3.iloc[j,0],bus3.iloc[j,1])
        
        if haversine(레지스,버스) <= 0.3:
            regis2.iloc[i,3]=regis2.iloc[i,3]+1
            
        if haversine(레지스,버스) <= 0.5:
            regis2.iloc[i,4]=regis2.iloc[i,4]+1

In [ ]:
from haversine import haversine

지하철역=(X값,Y값)
목표위도경도=(X값,Y값)
haversine(지하철역,목표위도경도)

In [543]:
# from haversine import haversine
# for i in range(0,len(regis2)):
#     for j in range(0,len(bus3)):
#         레지스=(regis2.iloc[i,1],regis2.iloc[i,2])
#         버스=(bus3.iloc[j,0],bus3.iloc[j,1])
#         if haversine(레지스,버스) <= 0.5:
#             regis2.iloc[i,4]=regis2.iloc[i,4]+1

In [544]:
regis2

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m
10,동대문역사문화공원,37.565138,127.007896,14,24
17,신도림,37.508725,126.891295,12,23
21,왕십리,37.561533,127.037732,14,35
25,시청,37.564718,126.977108,15,44
27,교대,37.493415,127.014080,11,18
32,강변,37.535095,127.094681,11,15
39,을지로3가,37.566295,126.991910,6,24
42,홍대입구,37.557192,126.925381,10,29
51,영등포구청,37.524970,126.895951,8,24
58,을지로4가,37.566941,126.998079,9,26


In [545]:
regis2['bikestation300m'] = 0
regis2['bikestation500m'] = 0

In [546]:
regis2.tail()

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m,bikestation300m,bikestation500m
339,구로디지털단지,37.485266,126.901401,15,30,0,0
344,문래,37.517933,126.894760,10,28,0,0
354,신촌,37.555134,126.936893,19,32,0,0
355,이대,37.556733,126.946013,16,31,0,0
356,아현,37.557345,126.956141,14,30,0,0


In [547]:
bike = pd.read_csv('seoulbike.csv')
bike.tail()

,번호,대여소,거치대수,주소,위도,경도
1918,969,은평 지웰테라스,9,서울특별시 은평구 진관4로 78-38 은평 지웰테라스,37.643860,126.931877
1919,971,역촌 센트레빌 아파트,10,서울특별시 은평구 갈현로3나길 23 역촌 센트레빌 아파트 107동 인근,37.602329,126.906548
1920,972,수색역,20,서울특별시 은평구 수색로 261 수색역,37.582161,126.894928
1921,9996,시설2,1,서울특별시 성동구 청계천로 540 서울시설공단 공공자전거 운영처,0.000000,0.000000
1922,0,위트콤공장,66,서울특별시 서초구 방배로 110 석교빌딩 4층,0.000000,0.000000


In [548]:
from haversine import haversine
for i in range(0,len(regis2)):
    for j in range(0,len(bike)):
        레지스=(regis2.iloc[i,1],regis2.iloc[i,2])
        바이크=(bike.iloc[j,4],bike.iloc[j,5])
        if haversine(레지스,바이크) <= 0.3:
            regis2.iloc[i,5]=regis2.iloc[i,5]+1
        
        if haversine(레지스,바이크) <= 0.5:
            regis2.iloc[i,6]=regis2.iloc[i,6]+1

In [ ]:
from haversine import haversine

지하철역=(X값,Y값)
목표위도경도=(X값,Y값)
haversine(지하철역,목표위도경도)

In [549]:
regis2

,역명,X좌표(WGS),Y좌표(WGS),busstation300m,busstation500m,bikestation300m,bikestation500m
10,동대문역사문화공원,37.565138,127.007896,14,24,3,7
17,신도림,37.508725,126.891295,12,23,3,4
21,왕십리,37.561533,127.037732,14,35,3,7
25,시청,37.564718,126.977108,15,44,5,10
27,교대,37.493415,127.014080,11,18,2,2
32,강변,37.535095,127.094681,11,15,3,4
39,을지로3가,37.566295,126.991910,6,24,2,11
42,홍대입구,37.557192,126.925381,10,29,4,6
51,영등포구청,37.524970,126.895951,8,24,1,6
58,을지로4가,37.566941,126.998079,9,26,2,8


In [550]:
regis2.rename(columns={'X좌표(WGS)':'X좌표','Y좌표(WGS)':'Y좌표'},inplace=True)

In [551]:
regis2.tail()

,역명,X좌표,Y좌표,busstation300m,busstation500m,bikestation300m,bikestation500m
339,구로디지털단지,37.485266,126.901401,15,30,4,5
344,문래,37.517933,126.894760,10,28,3,5
354,신촌,37.555134,126.936893,19,32,5,12
355,이대,37.556733,126.946013,16,31,4,5
356,아현,37.557345,126.956141,14,30,3,6


In [555]:
regis2['자전거대여소번호']=0

In [556]:
regis2

,역명,X좌표,Y좌표,busstation300m,busstation500m,bikestation300m,bikestation500m,자전거대여소번호
10,동대문역사문화공원,37.565138,127.007896,14,24,3,7,0
17,신도림,37.508725,126.891295,12,23,3,4,0
21,왕십리,37.561533,127.037732,14,35,3,7,0
25,시청,37.564718,126.977108,15,44,5,10,0
27,교대,37.493415,127.014080,11,18,2,2,0
32,강변,37.535095,127.094681,11,15,3,4,0
39,을지로3가,37.566295,126.991910,6,24,2,11,0
42,홍대입구,37.557192,126.925381,10,29,4,6,0
51,영등포구청,37.524970,126.895951,8,24,1,6,0
58,을지로4가,37.566941,126.998079,9,26,2,8,0


In [640]:
regis2.to_csv("regis2.csv",index=False,encoding="euc-kr")

In [638]:
seoulbike

,대여소번호,대여소,거치대수,주소,위도,경도
0,1001,광진교 남단 사거리(디지털프라자앞),15,서울특별시 강동구 올림픽로 674 로석빌딩,37.541794,127.124748
1,1002,해공공원(천호동),10,서울특별시 강동구 올림픽로 702 해공도서관,37.545261,127.125946
2,1003,해공도서관앞,20,서울특별시 강동구 올림픽로 702 해공도서관,37.543957,127.125488
3,1004,삼성광나루아파트 버스정류장,10,서울특별시 강동구 올림픽로 812,37.553329,127.128868
4,1006,롯데캐슬 115동앞,15,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트,37.554867,127.142799
...,...,...,...,...,...,...
1918,969,은평 지웰테라스,9,서울특별시 은평구 진관4로 78-38 은평 지웰테라스,37.643860,126.931877
1919,971,역촌 센트레빌 아파트,10,서울특별시 은평구 갈현로3나길 23 역촌 센트레빌 아파트 107동 인근,37.602329,126.906548
1920,972,수색역,20,서울특별시 은평구 수색로 261 수색역,37.582161,126.894928
1921,9996,시설2,1,서울특별시 성동구 청계천로 540 서울시설공단 공공자전거 운영처,0.000000,0.000000


In [566]:
seoulbike2 = seoulbike[['번호','위도','경도']]
seoulbike2.to_csv('seoulbike위도경도.csv',header=False,index=False)

In [567]:
seoulbike2

,번호,위도,경도
0,1001,37.541794,127.124748
1,1002,37.545261,127.125946
2,1003,37.543957,127.125488
3,1004,37.553329,127.128868
4,1006,37.554867,127.142799
...,...,...,...
1918,969,37.643860,126.931877
1919,971,37.602329,126.906548
1920,972,37.582161,126.894928
1921,9996,0.000000,0.000000


In [571]:
regis3=regis2[['역명','X좌표','Y좌표']]
regis3.to_csv('subway위도경도.csv',header=False,index=False)

In [568]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6850-4FA4

 C:\dss13\회귀분석 프로젝트 디렉터리

2020-06-16  오후 04:43    <DIR>          .
2020-06-16  오후 04:43    <DIR>          ..
2020-06-16  오전 11:57    <DIR>          .ipynb_checkpoints
2016-07-19  오후 02:05            86,180 2015day_1.csv
2020-06-11  오후 08:13        13,017,253 2015sub.csv
2017-03-22  오후 09:48            87,036 2016day_1.csv
2020-06-11  오후 08:13        13,055,897 2016sub.csv
2018-02-02  오후 12:38            87,067 2017day_1.csv
2020-06-11  오후 08:13        12,684,562 2017sub.csv
2019-02-03  오후 07:06            87,083 2018day_1.csv
2020-06-11  오후 08:13        11,462,196 2018sub.csv
2020-06-15  오후 05:20           636,859 20190710기준_서울시정류장정보(수정) (1).xlsx
2020-05-22  오전 10:15            87,003 2019day_1.csv
2020-06-11  오후 08:13        11,470,440 2019sub.csv
2020-06-15  오후 02:44           105,983 4647,4661번 서울교통공사 역별 주소 현황 및 전화번호.xlsx
2020-06-15  오후 02:11         2,999,104 air.csv
2020-06-15  오후 11:23         3,464,396 air_all.csv
2020-06-15  오후 02:1

In [560]:
dict1 = {}
dict1

{}

In [558]:
from haversine import haversine
dict1={}
for i in range(0,len(regis2)):
    for j in range(0,len(bike)):
        레지스=(regis2.iloc[i,1],regis2.iloc[i,2])
        바이크=(bike.iloc[j,4],bike.iloc[j,5])
        
        if haversine(레지스,바이크) <= 0.5:
            list

In [4]:
sub2 = pd.read_csv('sub2.csv',encoding='euc-kr')

In [5]:
sub2

,날짜,역명,구분,year,month,요일,17to20,holi,ctransfer
0,2016-01-01,시청,승차,2016,1,금,2752,True,1
1,2016-01-01,시청,하차,2016,1,금,1540,True,1
2,2016-01-01,을지로입구,하차,2016,1,금,4409,True,0
3,2016-01-01,을지로입구,승차,2016,1,금,7366,True,0
4,2016-01-01,을지로3가,승차,2016,1,금,1515,True,1
...,...,...,...,...,...,...,...,...,...
125641,2019-12-31,이대,하차,2019,12,화,3361,False,0
125642,2019-12-31,아현,승차,2019,12,화,1998,False,0
125643,2019-12-31,아현,하차,2019,12,화,2858,False,0
125644,2019-12-31,충정로,승차,2019,12,화,3218,False,1


In [611]:
seoulbike = pd.read_csv('seoulbike.csv',encoding='utf-8')

seoulbike

In [621]:
seoulbike.rename(columns={'번호':'대여소번호'},inplace=True)

seoulbike



seoulbike

bike_count = pd.read_csv('bike_count.csv',encoding='euc-kr')



bike_count

seoulbike[seoulbike.duplicated()==True]

seoulbike.head(40)

seoulbike.dtypes

bike_count.dtypes

a= pd.merge(bike_count,seoulbike,how='left',on='대여소번호')

len(seoulbike.대여소번호)

len(seoulbike.대여소번호.unique())

seoulbike=seoulbike.drop_duplicates()

len(seoulbike.대여소번호)

a= pd.merge(bike_count,seoulbike,how='left',on='대여소번호')

bike_count

a



bike_count